# Embeddings con OpenAI
Embeddings es un proceso mediante el cual se utiliza alguna tecnica/algoritmo que sea capaz de convertir palabras o texto a vectores de N dimensiones. Estos vectores contienen cierto nivel de información semantica sobre el texto o palabra. Por ejemplo, palabras que son muy similares van a tener valores cercanos en sus representaciones en vectores.

Hay varios modelos que son capaces de hacer un embedding de nuestro texto, en este cuaderno estaremos utilizando el embedding de OpenAI, el cual tiene la capacidad de posicionas muy bien palabras o textos segun su semantica. Este es el mismo embedding que utiliza GPT3. En este cuaderno estarás haciendo embedding de un texto para despues poder buscar cosas dentro de este texto por medio de preguntas en lenguaje natural.

In [11]:
# Importamos todas las dependencias requeridas, en este caso será Gradio para desarrollar la interfaz grafica y openai para realizar los llamados a su API 
import gradio as gr
import openai
import pandas as pd

from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity

ModuleNotFoundError: No module named 'gradio'

In [2]:
# Definimos la API Key para vincular el cuaderno con nuestra cuenta de OpenAI
openai.api_key = "sk-"

# Que es y cómo usar embeddings
Al hacer embedding de un dato, lo estamos convirtiendo a un vector numérico, datos similares estarán más cercanos entre si cuando semanticamente son similares

In [3]:
# Se puede hacer embeeding de palabras o cadenas de texto
palabras = ["casa", "perro", "gato", "lobo", "leon", "zebra", "tigre"]

In [4]:
diccionario = {}
for i in palabras:
    diccionario[i] = get_embedding(i, engine="text-embedding-ada-002")
    

In [6]:
diccionario.keys()

dict_keys(['casa', 'perro', 'gato', 'lobo', 'leon', 'zebra', 'tigre'])

In [9]:
palabra = "gato"
print("Primeros 10 valores de {}:\n".format(palabra), diccionario[palabra][:10])
print("\n")
print("Número de dimensiones del dato embebido\n", len(diccionario[palabra]))


Primeros 10 valores de gato:
 [-0.024576973170042038, 0.009653845801949501, -0.0003061550669372082, -0.011156566441059113, -0.007181836757808924, 0.01183311641216278, -0.022013889625668526, -0.024954279884696007, -0.0052172401919960976, -0.019047480076551437]


Número de dimensiones del dato embebido
 1536


## Comparar dos embeddings
Debido a que los embeddings son una representacion vectorial de los datos en un espacio latente, podemos medir la distancia entre dos vectores y asi obtener que tan similares son. Podemos comparar una palabra nueva o alguna de las que ya fueron embebidas 
OJO: No necesariamente es similitud al objeto. Ej. perro y gato aun siendo "opuestos" semanticamente estan cerca pues tienen una relación.

In [13]:
n_palabra = "agujero negro" # Palabra nueva a comparar
palabra_comparar = "perro" # Palabra del diccionario con la que compararemos la nueva palabra
n_palabra_embed = get_embedding(n_palabra, engine="text-embedding-ada-002")
similitud = cosine_similarity(diccionario[palabra_comparar], n_palabra_embed)
print(similitud)

0.8159836735319491


# Sumar embeddings
Como los vectores contienen valores numericos, podemos sumarlos y el resultado será un nuevo vector de un concepto que una los elementos sumados

In [14]:
# Suma dos listas usando pandas
sumados = (pd.DataFrame(diccionario["leon"])) + (pd.DataFrame(diccionario["zebra"]))
len(sumados)

for key, value in diccionario.items():
    print(key, ":", cosine_similarity(diccionario[key], sumados))


casa : [0.82819415]
perro : [0.85046401]
gato : [0.85630059]
lobo : [0.85639036]
leon : [0.9531031]
zebra : [0.95310309]
tigre : [0.88060081]


# Aplicacion de un Chatbot

Usaremos Gradio para hacer una interfaz básica donde podremos hacer preguntas y obtendremos una respuesta. 
Para esto reutilizaremos lo que hemos visto hasta el momento pero usaremos el archivo de **chatbot_qa.csv**

In [16]:
def embed_text(path="texto.csv"):
    conocimiento_df = pd.read_csv(path)
    conocimiento_df['Embedding'] = conocimiento_df['texto'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
    conocimiento_df.to_csv('embeddings.csv')
    return conocimiento_df

def buscar(busqueda, datos, n_resultados=5):
    busqueda_embed = get_embedding(busqueda, engine="text-embedding-ada-002")
    datos["Similitud"] = datos['Embedding'].apply(lambda x: cosine_similarity(x, busqueda_embed))
    datos = datos.sort_values("Similitud", ascending=False)
    return datos.iloc[:n_resultados][["texto", "Similitud", "Embedding"]]

texto_emb = embed_text("./chatbot_qa.csv")

with gr.Blocks() as demo:
    busqueda = gr.Textbox(label="Buscar")
    output = gr.DataFrame(headers=['texto'])
    greet_btn = gr.Button("Preguntar")
    greet_btn.click(fn=buscar, inputs=[busqueda, gr.DataFrame(texto_emb)], outputs=output)

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# Procesar datos de un PDF
Haremos ahora un ejemplo donde leemos un PDF para poder hacer preguntas y traer un exctracto del PDF

In [1]:
# pip install langchain pypdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("doc/I-CP-01 (A).pdf")
pages = loader.load_and_split()

In [2]:
# Un elemento por cada página
pages[3].page_content

'SIN CLASIFICAR \n1-CP-01 (A) \nEl Almirante de la Flota \nFebrero 2015 \nORDEN DE PROMULGACION \nl. Conforme a lo dispuesto en la Instrucción Permanente de Organización núm. \n10/2010 sobre la DOCTRINA EN LA ARMADA (CAMBIO 3), se declara \nreglamentaria la publicación I-CP-0 1 (A) "TÉCNICAS Y MATERIALES DE \nCONTRAINCENDIOS". \n2. La I-CP-Ol(A) es una publicación SIN CLASIFICAR y entrará en vigor a su \nrecepción. \n3. La I-CP-Ol(A) anula y sustituye a la publicación I-CP-01 (CAMBIO 1), que se \ndestruirá de acuerdo a las disposiciones en vigor. \n--------\n111 ORIGINAL \nSIN CLASU=ICAR'

In [3]:
# Objeto que va a hacer los cortes en el texto
split = CharacterTextSplitter(chunk_size=300, separator = '.\n')

In [4]:
textos = split.split_documents(pages) # Lista de textos

Created a chunk of size 1312, which is longer than the specified 300
Created a chunk of size 1803, which is longer than the specified 300


In [5]:
print(textos[0].page_content)
#print(textos[0])

SIN CLAS IFICAR  
 
SIN CLAS IFICAR  
  
 
 
 
 
 
I-CP-01 (A) 
 
  
TECNICAS Y  
MATERIALES DE 
CONTRAINCENDIOS


In [6]:
import pandas as pd

# Extraemos la parte de page_content de cada texto y lo pasamos a un dataframe
textos = [str(i.page_content) for i in textos] #Lista de parrafos
parrafos = pd.DataFrame(textos, columns=["texto"])
print(parrafos)

                                                 texto
0    SIN CLAS IFICAR  \n \nSIN CLAS IFICAR  \n  \n ...
1    SIN CLAS IFICAR  \nI-CP-01 (A) \n   \n \n  \nO...
2    SIN CLAS IFICAR  \nI-CP-01 (A) \n   \n \n  \nO...
3    SIN CLASIFICAR \n1-CP-01 (A) \nEl Almirante de...
4    SIN CLASIFICAR \n1-CP-01 (A) \nPÁGINA EN BLANC...
..                                                 ...
318  SIN CLAS IFICAR  \nI-CP-01(A) \n   \n  \nORIGI...
319  SIN CLAS IFICAR  \nI-CP-01(A) \n   \n  \nORIGI...
320  SIN CLAS IFICAR  \nI-CP-01(A)  \n \nORIGINAL  ...
321  SIN CLAS IFICAR  \nI-CP-01(A)  \n \nORIGINAL  ...
322  SIN CLAS IFICAR  \nSIN CLASIFICAR   \n        ...

[323 rows x 1 columns]


In [23]:
import tensorflow_hub as hub
import pandas as pd
import tensorflow as tf
import os

# Cargar el modelo Universal Sentence Encoder (versión 4)
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Obtener embeddings para los párrafos
try:
    embeddings = embed(parrafos["texto"])
except Exception as e:
    print(f"Error al obtener embeddings: {e}")
    exit()

# Agregar los embeddings al DataFrame
parrafos['Embedding'] = tf.convert_to_tensor(embeddings).numpy().tolist()

# Guardar el DataFrame con los embeddings en un archivo CSV (verifica si existe)
nombre_archivo = "emb-SI.csv"
if os.path.exists(nombre_archivo):
    nombre_archivo = f"{nombre_archivo}-{int(time.time())}"

parrafos.to_csv(nombre_archivo, index=False)



In [33]:
# La misma funcion del chatbot de pregunts y respuestas
def embed_text(path="texto.csv"):
    conocimiento_df = pd.read_csv(path)
    conocimiento_df['Embedding'] = conocimiento_df['texto'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
    conocimiento_df.to_csv('mtg-embeddings.csv')
    return conocimiento_df

def buscar(busqueda, datos, n_resultados=5):
    busqueda_embed = get_embedding(busqueda, engine="text-embedding-ada-002")
    datos["Similitud"] = datos['Embedding'].apply(lambda x: cosine_similarity(x, busqueda_embed))
    datos = datos.sort_values("Similitud", ascending=False)
    return datos.iloc[:n_resultados][["texto", "Similitud", "Embedding"]]

texto_emb = parrafos
with gr.Blocks() as demo:
    busqueda = gr.Textbox(label="Buscar")
    output = gr.DataFrame(headers=['texto'])
    greet_btn = gr.Button("Preguntar")
    greet_btn.click(fn=buscar, inputs=[busqueda, gr.DataFrame(texto_emb)], outputs=output)

demo.launch()


# resp = buscar("Con cuanta vida empiezo?", parrafos, 5) # Reutilizamos la funcion de "buscar" del app de gradio

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [ ]:
# resp = buscar("Con cuanta vida empiezo?", parrafos, 5) # Reutilizamos la funcion de "buscar" del app de gradio
# print(resp.texto)